In [1]:
# dependencies
from langchain_openai import ChatOpenAI ## for openAI
from langchain_google_genai import ChatGoogleGenerativeAI ## for gemini
from langchain_google_vertexai import VertexAI, ChatVertexAI, VertexAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.tools import tool
from pinecone.grpc import PineconeGRPC as pinecone_grpc
from langchain_core.output_parsers import JsonOutputParser
import os
from dotenv import load_dotenv
import vertexai


In [2]:
load_dotenv() # load environment variables
vertexai.init(project=os.getenv("PROJECT_ID"), location=os.getenv("PROJECT_LOCATION"))

In [3]:
# llm = ChatVertexAI(model="gemini-1.5-pro-001") ## llm for the agent
llm= ChatOpenAI(model="gpt-4o",temperature=0)

In [4]:
llm.invoke("what is your name and which company made you in no more than 10 words").content

"I'm ChatGPT, created by OpenAI."

In [5]:
class PineconeClass:
  try:

    def __init__(self,user_id:str) -> None:
      self.user_id = user_id
      self.pc = pinecone_grpc(api_key=os.getenv("PINECONE_API_KEY"))
      ## pinecone db index
      self.index = self.pc.Index(os.getenv("PINECONE_INDEX_NAME"))
    
    def insert_vectors(self,images_vector:list[dict])->int:
      upsert_response = self.index.upsert(
        vectors=images_vector,
        namespace=self.user_id,
      )
      return upsert_response.upserted_count ## returns the total number of vectors inserted/updated

    def fetch_similar_vectors(self, vector_list:list[float|int], top_k:int=3,include_values:bool=False,filter:dict={},include_metadata:bool=False):
      similar_vectors = self.index.query(
        namespace=self.user_id,
        vector=vector_list,
        filter=filter,
        top_k=top_k,
        include_values=include_values,
        include_metadata=include_metadata
      )
      return similar_vectors
    
  except Exception as e:
    raise Exception(f"Error in PineconeClass: {str(e)}")
  


In [6]:
from pydantic import BaseModel, Field

class RecommendedOutfit(BaseModel):
  upperwear_url:str = Field( title="upperwear_url",description="the url from the upperwear collection if the user asked for upperwear recommendation which when paired with the lowerwear collection will give the best outfit. Returns 'none' if the user did not ask for upperwear recommendation")
  lowerwear_url:str = Field(title="lowerwear_url",description="the url from the lowerwear collection if the user asked for lowerwear recommendation which when paired with the upperwear collection will give the best outfit. Returns 'none' if the user did not ask for lowerwear recommendation")

In [7]:
# for retrieving the clothes of upperwear tag
@tool
def retrieve_upperwear(user_prompt_vector:list[float])->list[dict]:
  """fetches the top 3 upperwear vectors closest to the vector of the user_prompt_vector from the pinecone db vector store"""
  print("GETTING THE UPPERWEAR COLLECTION")
  pinecone_class_instance = PineconeClass(user_id="JKVDl1ErPjaj3TPRNuBUsN3W9xS2")
  
  upperwear_clothes_data = pinecone_class_instance.fetch_similar_vectors(vector_list=user_prompt_vector, top_k=4, filter={"tag":"upperwear"},include_metadata=True)["matches"] ## list of dicts
  upperwear_collection=[{"type":"image_url","image_url":data["metadata"]["url"]} for data in upperwear_clothes_data]
  return upperwear_collection

# for retrieving the clothes of lowerwear tag
@tool
def retrieve_lowerwear(user_prompt_vector:list[float])->list[dict]:
  """fetches the 3 lowerwear vectors closest to the vector of the user_prompt_vector from the pinecone db vector store"""
  print("GETTING THE LOWERWEAR COLLECTION")
  pinecone_class_instance = PineconeClass(user_id="JKVDl1ErPjaj3TPRNuBUsN3W9xS2")
  lowerwear_collection=[]
  lowerwear_clothes_data = pinecone_class_instance.fetch_similar_vectors(vector_list=user_prompt_vector, top_k=4, filter={"tag":"lowerwear"},include_metadata=True)["matches"] ## list of dicts
  
  lowerwear_collection=[{"type":"image_url","image_url":data["metadata"]["url"]} for data in lowerwear_clothes_data]
  return lowerwear_collection

@tool
def get_recommendation(user_prompt:str, upperwear_data,lowerwear_data)->dict:
  """will get the data of the upperwear and lowerwear after the data has been fetched from pinecone vector store db and generate the recommendation based on that data using the user_prompt"""
  print("GENERATING  THE RECOMMENDATION")
  model = VertexAI(model_name="gemini-1.5-flash",temperture=2, top_p=0.95) ## Model for generating the output

  parser = JsonOutputParser(pydantic_object=RecommendedOutfit) ## Parser for parsing the output
  
  prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful bot that helps users answer questions based on the user input. You have to respond according to the {format_instructions}. In addition to the instructions, if the accessibility_type is blind then you have to provide the output explaining each image like how it feels so that the blind person can identify the cloth by feeling it. If the accessibility_type is some kind of color blind disorder then you have to explain the cloth in the color same as the color blind person can see."),
    ("human", "{user_prompt}. The uppperwear data is {upperwear_data} and the lowerwear data is {lowerwear_data}.")
  ])

  chain = prompt | model | parser
  recommendation = chain.invoke({
    "format_instructions":parser.get_format_instructions(),
    "upperwear_data":upperwear_data,
    "lowerwear_data":lowerwear_data,
    "user_prompt":user_prompt
  })
  return recommendation

@tool
def get_text_vector(user_prompt:str)->list[float]:
  """Will accept the user input and will return the vector of the user input"""
  print("GENERATING THE EMBEDDINGS")
  embedding_model = VertexAIEmbeddings(model_name="multimodalembedding")
  user_prompt_vector = embedding_model.embed_query(text=user_prompt)
  print("USER PROMPT VECTOR",user_prompt_vector)
  return user_prompt_vector

In [8]:
prompt = ChatPromptTemplate.from_messages(
    [
      ("system", """
      You are a powerful AI bot whose job is to recommend the outfits to the user. You have to understand the user_prompt and then use the  tools provided to get the result. If the user_prompt involves a description of a clothing item that belongs to the category of upperwear you will retrieve the results for lowerwear. If the user_prompt involves the description of a clothing item that belongs to the lowerwear category you will fetch the results from the upperwear category. If the user_prompt does not mention any of the above you will get the results from both the categories
      """),
      ("human", "{user_prompt}."),
      ("placeholder","{agent_scratchpad}")
    ]
  )

In [9]:
tools=[retrieve_upperwear,retrieve_lowerwear,get_recommendation,get_text_vector]


In [10]:
agent = create_tool_calling_agent(prompt=prompt,llm=llm,tools=tools)

In [11]:
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)


In [12]:
agent_executor.invoke({
  "user_prompt":"what should I wear for the beach party today evening in Toronto?",
  "format_instructions":
})

SyntaxError: expression expected after dictionary key and ':' (581349981.py, line 3)

In [129]:
llm = ChatVertexAI(model_name="gemini-1.5-pro-001")

In [130]:
llm.invoke("what is your name and which company made you in no more than 10 words and do you support function calling").content

I0000 00:00:1722133802.409219  985370 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


'I am Bard, a large language model created by Google AI. I support function calling. \n'